In [15]:
%pip install lyricsgenius
%pip install spotipy
from lyricsgenius import Genius
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import numpy as np
# import seaborn as sns
import requests


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [19]:
cid = "6540d0625e24409ea4f83ff44c9ed3b3"
secret = "81c6110978424fe3be0e23edda134f70"
# genius_key = "6mOCQw6G-wXsiEqXgiWa54JYyZXqNo3B9oOFBVli96oO2girdJVYcnzmesC_p5vy"
genius_key = "-svGbhXwBRww9Fwqb1jx2C0evM4BpO9aZWQL6vfG45oF_zPaimZP7cCWkOzVfnSL"

#Authentication - without user
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

# playlist_link = "https://open.spotify.com/playlist/37i9dQZEVXbNG2KDcFcKOF?si=1333723a6eff4b7f"
# playlist_link = "https://open.spotify.com/playlist/5MGi3muFW4A7exL52WaFua?si=c1f91ee9ee044266" # MY DG PLAYLIST
playlist_link = "https://open.spotify.com/playlist/0bGQUEAot6XMESsV4WbDkW?si=9887dbba8f3e46a4" # ALL DG Songs
playlist_URI = playlist_link.split("/")[-1].split("?")[0]
track_uris = [x["track"]["uri"] for x in sp.playlist_tracks(playlist_URI)["items"]]

In [20]:
class GetLyrics():
    
    def __init__(self, spotify_client_id, spotify_client_secret, playlist_id, genius_key):
        self.spotify_client_id = spotify_client_id
        self.spotify_client_secret = spotify_client_secret
        # self.user_id = user_id
        self.playlist_id = playlist_id
        self.genius_key = genius_key
        
    def get_playlist_info(self):
        token = SpotifyClientCredentials(client_id=self.spotify_client_id, client_secret=self.spotify_client_secret).get_access_token()
        sp = spotipy.Spotify(token)
        playlist = sp.playlist_tracks(self.playlist_id)
        self.playlist = playlist
        return self.playlist
    
    def get_track_names(self):
        track_names = []
        for song in range(len(self.playlist['items'])):
            track_names.append(self.playlist['items'][song]['track']['name'])
        self.track_names = track_names
        return self.track_names
    
    def get_track_artists(self):
        track_artists = []
        for song in range(len(self.playlist['items'])):
            track_artists.append(self.playlist['items'][song]['track']['artists'][0]['name'])
        self.track_artists = track_artists
        return self.track_artists
        
    def request_song_info(self, track_name, track_artist):
        self.track_name = track_name
        self.track_artist = track_artist
        base_url = 'https://api.genius.com'
        headers = {'Authorization': 'Bearer ' + self.genius_key}
        search_url = base_url + '/search'
        data = {'q': track_name + ' ' + track_artist}
        response = requests.get(search_url, data=data, headers=headers)
        self.response = response
        return self.response

    def check_hits(self):
        json = self.response.json()
        remote_song_info = None
        for hit in json['response']['hits']:
            if self.track_artist.lower() in hit['result']['primary_artist']['name'].lower():
                remote_song_info = hit
                break
        self.remote_song_info = remote_song_info
        return self.remote_song_info
    
    def get_url(self):
        song_url = self.remote_song_info['result']['url']
        self.song_url = song_url
        return self.song_url
    
    def scrape_lyrics(self):
        page = requests.get(self.song_url)
        html = BeautifulSoup(page.text, 'html.parser')
        lyrics1 = html.find("div", class_="lyrics")
        lyrics2 = html.find("div", class_="Lyrics__Container-sc-1ynbvzw-2 jgQsqn")
        if lyrics1:
            lyrics = lyrics1.get_text()
        elif lyrics2:
            lyrics = lyrics2.get_text()
        elif lyrics1 == lyrics2 == None:
            lyrics = None
        return lyrics

    def get_lyrics(self):
        playlist = GetLyrics.get_playlist_info(self)
        track_names = GetLyrics.get_track_names(self)
        track_artists = GetLyrics.get_track_artists(self)
        song_lyrics = []
        for i in range(len(self.track_names)):
            print("\n")
            print(f"Working on track {i}.")
            response = GetLyrics.request_song_info(self, self.track_names[i], self.track_artists[i])
            remote_song_info = GetLyrics.check_hits(self)
            if remote_song_info == None:
                lyrics = None
                print(f"Track {i} is not in the Genius database.")
            else:
                url = GetLyrics.get_url(self)
                lyrics = GetLyrics.scrape_lyrics(self)
                if lyrics == None:
                    print(f"Track {i} is not in the Genius database.")
                else:
                    print(f"Retrieved track {i} lyrics!")
            song_lyrics.append(lyrics)
        return song_lyrics

In [21]:
songs = GetLyrics(cid, secret, playlist_URI, genius_key)
song_lyrics = songs.get_lyrics()

/var/folders/0q/7qqchjfd3h7gd1zmyqrf1rxw0000gn/T/ipykernel_52925/3684460710.py:11: DeprecationWarning: You're using 'as_dict = True'.get_access_token will return the token string directly in future versions. Please adjust your code accordingly, or use get_cached_token instead.
  token = SpotifyClientCredentials(client_id=self.spotify_client_id, client_secret=self.spotify_client_secret).get_access_token()
HTTP Error for GET to https://api.spotify.com/v1/playlists/0bGQUEAot6XMESsV4WbDkW/tracks with Params: {'limit': 100, 'offset': 0, 'fields': None, 'market': None, 'additional_types': 'track'} returned 400 due to Only valid bearer authentication supported


SpotifyException: http status: 400, code:-1 - https://api.spotify.com/v1/playlists/0bGQUEAot6XMESsV4WbDkW/tracks?limit=100&offset=0&additional_types=track:
 Only valid bearer authentication supported, reason: None

In [22]:
%pip install korean_romanizer
from korean_romanizer.romanizer import Romanizer

r = Romanizer("안녕하세요")
r.romanize() 

  Obtaining dependency information for korean_romanizer from https://files.pythonhosted.org/packages/e7/97/5029e26656a916c03c714d5ddac1fd48b0c39a9ce2b3e027ec1af144df45/korean_romanizer-0.25.1-py3-none-any.whl.metadata

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


'annyeonghaseyo'

In [23]:
token = "-svGbhXwBRww9Fwqb1jx2C0evM4BpO9aZWQL6vfG45oF_zPaimZP7cCWkOzVfnSL"

genius = Genius(token)
# artist = genius.search_artist('Astro')
song = genius.search_song("Run", "BTS")
# pd.DataFrame(song.lyrics()).to_csv('Lyrics.csv')
print(song.lyrics)

Searching for "Run" by BTS...
Done.
32 ContributorsTranslationsУкраїнськаEspañolFrançaisEnglishRomanizationRUN Lyrics[방탄소년단 "RUN" 가사]

[Verse 1: RM]
넌 내 하나뿐인 태양 세상에 딱 하나
널 향해 피었지만 난 자꾸 목말라
너무 늦었어 늦었어 너 없이 살 순 없어
가지가 말라도 더 힘껏 손을 뻗어

[Verse 2: SUGA]
손 뻗어봤자 금세 깨버릴 꿈 (꿈, 꿈)
미칠 듯 달려도 또 제자리일 뿐 (뿐, 뿐)
그냥 날 태워줘 그래 더 밀쳐내줘
이건 사랑에 미친 멍청이의 뜀박질

[Pre-Chorus: V]
더 뛰게 해줘
나를 더 뛰게 해줘
두 발에 상처만 가득해도
니 얼굴만 보면 웃는 나니까

[Chorus: Jung Kook, Jimin]
다시 run, run, run, 난 멈출 수가 없어
또 run, run, run, 난 어쩔 수가 없어
어차피 이것밖에 난 못해
너를 사랑하는 것 밖엔 못해
다시 run, run, run, 넘어져도 괜찮아
또 run, run, run, 좀 다쳐도 괜찮아
가질 수 없다 해도 난 족해
바보 같은 운명아 나를 욕해
You might also like[Post-Chorus: Jung Kook, RM]
Run (Don't tell me, bye, bye)
Run (You make me cry, cry)
Run (Love is a lie, lie)
Don't tell me, don't tell me
Don't tell me bye bye

[Verse 3: j-hope]
다 끝난 거라는데 난 멈출 수가 없네
땀인지 눈물인지 나 더는 분간 못해, oh
내 발가벗은 사랑도 거친 태풍 바람도
나를 더 뛰게만 해 내 심장과 함께

[Pre-Chorus: Jin]
더 뛰게 해줘
나를 더 뛰게 해줘
두 발에 상처만 가득해도
니 얼굴만 보면 웃는 나니까

[Chorus: Jung Kook, Jimin]
다시 run, run, run,

In [25]:
print(Romanizer(song.lyrics).romanize())

32 ContributorsTranslationsУкраїнськаEspañolFrançaisEnglishRomanizationRUN Lyrics[bangtansonyeondan "RUN" gasa]

[Verse 1: RM]
neon nae hanappunin taeyang sesange ttak hana
neol hyanghae pieotjiman nan jakku mokmalra
neomu neujeosseo neujeosseo neo eopsi sal sun eopseo
gajiga malrado deo himkkeot soneul ppeodeo

[Verse 2: SUGA]
son ppeodeobwatja geumse kkaebeoril kkum (kkum, kkum)
michil deut dalryeodo tto jejariil ppun (ppun, ppun)
geunyang nal taewojwo geurae deo milchyeonaejwo
igeon sarange michin meongcheongiui ttwimbakjil

[Pre-Chorus: V]
deo ttwige haejwo
nareul deo ttwige haejwo
du bare sangcheoman gadeukhaedo
ni eolgulman bomyeon utneun nanikka

[Chorus: Jung Kook, Jimin]
dasi run, run, run, nan meomchul suga eopseo
tto run, run, run, nan eojjeol suga eopseo
eochapi igeotbakke nan mothae
neoreul saranghaneun geot bakken mothae
dasi run, run, run, neomeojyeodo gwaenchana
tto run, run, run, jom dachyeodo gwaenchana
gajil su eopda haedo nan jokhae
babo gateun unmyeonga nareul yokh